<a href="https://colab.research.google.com/github/sean-otoole/HODD/blob/main/parkinsons_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!cd /content/drive/MyDrive/HODD/Kamath_et_al_2022/

/bin/bash: line 1: cd: /content/drive/MyDrive/HODD/Kamath_et_al_2022/: No such file or directory


In [7]:
import pandas as pd

tsv_path = '/content/drive/MyDrive/HODD/Kamath_et_al_2022/Homo_features.tsv'
df = pd.read_csv(tsv_path, sep='\t')
print(df.head())

print(df.shape)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/HODD/Kamath_et_al_2022/Homo_features.tsv'

In [10]:
!cd ..

In [11]:
ls

sample_data/
